In [1]:
!pip install scikit-learn

In [2]:
import os
import gc
import json
import random
import numpy as np
import pandas as pd
import nltk
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer, BartForConditionalGeneration,
    Seq2SeqTrainer, Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq, EarlyStoppingCallback
)
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer
from bert_score import score
from sklearn.model_selection import train_test_split
import re
import sqlite3
import optuna
import logging
import shutil
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner

In [3]:
# Setup logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

In [22]:
# Seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Check CUDA
logger.info(f"CUDA available: {torch.cuda.is_available()}")
logger.info(f"CUDA version: {torch.version.cuda}")
logger.info(f"CUDNN version: {torch.backends.cudnn.version()}")

# NLTK downloads
try:
    nltk.download('wordnet', quiet=True)
    nltk.download('omw-1.4', quiet=True)
    nltk.download('punkt', quiet=True)
    nltk.download('punkt_tab', quiet=True)
except Exception as e:
    logger.error(f"NLTK download failed: {e}")
    raise

# Directories
PROJECT_ROOT = r"D:\A_CSE499"
DATA_DIR = os.path.join(PROJECT_ROOT, "data_phase2")
OUTPUT_DIR = os.path.join(PROJECT_ROOT, "outputLarge_B_phase2")
MODEL_PATH = os.path.join(PROJECT_ROOT, "outputLarge_B", "final_model")
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Validate model path
if not os.path.exists(MODEL_PATH):
    logger.error(f"Model path does not exist: {MODEL_PATH}")
    raise FileNotFoundError(f"Model path does not exist: {MODEL_PATH}. Please ensure the model is saved or update MODEL_PATH.")

    
if os.path.exists(MODEL_PATH):
    print(f"✅ Model path exists: {MODEL_PATH}")
else:
    print(f"❌ Model path does NOT exist: {MODEL_PATH}")

2025-07-24 03:15:32,381 - INFO - CUDA available: True
2025-07-24 03:15:32,382 - INFO - CUDA version: 12.8
2025-07-24 03:15:32,383 - INFO - CUDNN version: 90701


✅ Model path exists: D:\A_CSE499\outputLarge_B\final_model


In [5]:
# Load tokenizer and set device
tokenizer = AutoTokenizer.from_pretrained(os.path.join(PROJECT_ROOT, "outputLarge_B", "final_model"))
device = torch.device("cuda:0")
if not torch.cuda.is_available():
    raise RuntimeError("CUDA is not available. Please check your PyTorch installation and NVIDIA drivers.")

In [6]:
# Punctuation cleaning function
def fix_punctuation_spacing(text):
    if not isinstance(text, str):
        return text
    text = text.replace(r'\newline', ' ')
    text = re.sub(r'\s+([,.;:!?])', r'\1', text)
    text = re.sub(r'([,.;:!?])([^\s\W])', r'\1 \2', text)
    text = re.sub(r'\(\s+', '(', text)
    text = re.sub(r'\s+\)', ')', text)
    text = re.sub(r'"\s+', '"', text)
    text = re.sub(r'\s+"', '"', text)
    text = re.sub(r"'\s+", "'", text)
    text = re.sub(r"\s+'", "'", text)
    text = re.sub(r'\s*[-–—]+\s*', ' — ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

In [7]:
# Preprocess function
def preprocess_function(batch):
    inputs = []
    targets = []
    for c, q in zip(batch['context'], batch['question']):
        c_clean = fix_punctuation_spacing(str(c)) if c else ""
        q_clean = fix_punctuation_spacing(str(q)) if q else ""
        if q_clean.startswith("What is the"):
            q_clean = q_clean.replace("What is the", "What can you tell about")
        if c_clean and q_clean:
            inputs.append(c_clean)
            targets.append(q_clean)

    model_inputs = tokenizer(
        inputs,
        max_length=64,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=64,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )
    model_inputs["labels"] = labels["input_ids"]
    model_inputs["labels"][model_inputs["labels"] == tokenizer.pad_token_id] = -100
    return model_inputs

In [8]:
# Load and validate dataset
def validate_csv(file_path, required_columns):
    if not os.path.exists(file_path):
        logger.error(f"CSV file not found: {file_path}")
        raise FileNotFoundError(f"CSV file not found: {file_path}")
    df = pd.read_csv(file_path)
    for col in required_columns:
        if col not in df.columns:
            logger.error(f"Column '{col}' missing in {file_path}")
            raise ValueError(f"Column '{col}' missing in {file_path}")
    return df

In [9]:
# Load combined 100k dataset
df = validate_csv(os.path.join(DATA_DIR, "final_100k_dataset.csv"), ['context', 'question'])

# Shuffle and reset index
df = df.sample(frac=1, random_state=SEED).reset_index(drop=True)

# Split into train (80%), validation (10%), test (10%)
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=SEED)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=SEED)

# Log dataset sizes
logger.info(f"Train dataset size: {len(train_df)}")
logger.info(f"Validation dataset size: {len(val_df)}")
logger.info(f"Test dataset size: {len(test_df)}")

2025-07-22 16:31:57,758 - INFO - Train dataset size: 80000
2025-07-22 16:31:57,759 - INFO - Validation dataset size: 10000
2025-07-22 16:31:57,759 - INFO - Test dataset size: 10000


In [10]:
sample = df.sample(5)
for i, row in sample.iterrows():
    print(f"Context: {row['context'][:300]}")
    print(f"Question: {row['question']}")
    # print(f"Answer: {row['answer']}")
    print("="*80)

Context: The big problem that you have in looking at polls of people who watched conventions is something called sorting . Republicans watch Republican conventions and Democrats watch Democratic conventions . The problem is probably even worse than that because it is usually the more hardcore in those groups
Question: Why do the republicans and democrats watch the conventions ?
Context: Peter Holmström: Peter Holmström is an American rock musician.  He is in the bands The Dandy Warhols, Pete International Airport, Radis Noir and Rebel Drones.  Holmström's first guitar was a Gibson SG.  Holmström is an amateur photographer.  In the past he has also composed music for his sister's th
Question: "Plan A" is a song by American rock band The Dandy Warhols, an American alternative rock band, formed in Portland, Oregon in which year, by singer-guitarist Courtney Taylor-Taylor and guitarist Peter Holmström?
Context: Strangers (Halsey song): "Strangers" is a song recorded by American singer and 

In [11]:
# After splitting your dataframes:
train_df = train_df
val_df = val_df
test_df = test_df

In [12]:
# Convert to HuggingFace Datasets
dataset_train = Dataset.from_pandas(train_df)
dataset_val = Dataset.from_pandas(val_df)
dataset_test = Dataset.from_pandas(test_df)

In [40]:
# Tokenize datasets
tokenized_dir = os.path.join(OUTPUT_DIR, "tokenized_datasets")
os.makedirs(tokenized_dir, exist_ok=True)

def save_tokenized_datasets(train_dataset, val_dataset, test_dataset):
    if not os.path.exists(os.path.join(tokenized_dir, "train")):
        train_dataset.save_to_disk(os.path.join(tokenized_dir, "train"))
        logger.info("Saved tokenized training dataset")
    if not os.path.exists(os.path.join(tokenized_dir, "val")):
        val_dataset.save_to_disk(os.path.join(tokenized_dir, "val"))
        logger.info("Saved tokenized validation dataset")
    if not os.path.exists(os.path.join(tokenized_dir, "test")):
        test_dataset.save_to_disk(os.path.join(tokenized_dir, "test"))
        logger.info("Saved tokenized test dataset")

def load_tokenized_datasets():
    global processed_train_dataset, processed_val_dataset, processed_test_dataset
    if os.path.exists(os.path.join(tokenized_dir, "train")):
        train_dataset = Dataset.load_from_disk(os.path.join(tokenized_dir, "train"))
        temp = train_dataset.filter(lambda x: all(k in x for k in ["input_ids", "attention_mask", "labels"]))
        if len(temp) == len(train_dataset):
            logger.info("Loaded tokenized training dataset")
        else:
            logger.warning("Invalid train dataset format, re-tokenizing...")
            train_dataset = dataset_train.map(preprocess_function, batched=True, batch_size=50, remove_columns=['context', 'question'])
            train_dataset.save_to_disk(os.path.join(tokenized_dir, "train"))
            logger.info("Tokenized and saved training dataset")
    else:
        train_dataset = dataset_train.map(preprocess_function, batched=True, batch_size=50, remove_columns=['context', 'question'])
        train_dataset.save_to_disk(os.path.join(tokenized_dir, "train"))
        logger.info("Tokenized and saved training dataset")

    if os.path.exists(os.path.join(tokenized_dir, "val")):
        val_dataset = Dataset.load_from_disk(os.path.join(tokenized_dir, "val"))
        temp = val_dataset.filter(lambda x: all(k in x for k in ["input_ids", "attention_mask", "labels"]))
        if len(temp) == len(val_dataset):
            logger.info("Loaded tokenized validation dataset")
        else:
            logger.warning("Invalid val dataset format, re-tokenizing...")
            val_dataset = dataset_val.map(preprocess_function, batched=True, batch_size=50, remove_columns=['context', 'question'])
            val_dataset.save_to_disk(os.path.join(tokenized_dir, "val"))
            logger.info("Tokenized and saved validation dataset")
    else:
        val_dataset = dataset_val.map(preprocess_function, batched=True, batch_size=50, remove_columns=['context', 'question'])
        val_dataset.save_to_disk(os.path.join(tokenized_dir, "val"))
        logger.info("Tokenized and saved validation dataset")

    if os.path.exists(os.path.join(tokenized_dir, "test")):
        test_dataset = Dataset.load_from_disk(os.path.join(tokenized_dir, "test"))
        temp = test_dataset.filter(lambda x: all(k in x for k in ["input_ids", "attention_mask", "labels"]))
        if len(temp) == len(test_dataset):
            logger.info("Loaded tokenized test dataset")
        else:
            logger.warning("Invalid test dataset format, re-tokenizing...")
            test_dataset = dataset_test.map(preprocess_function, batched=True, batch_size=50, remove_columns=['context', 'question'])
            test_dataset.save_to_disk(os.path.join(tokenized_dir, "test"))
            logger.info("Tokenized and saved test dataset")
    else:
        test_dataset = dataset_test.map(preprocess_function, batched=True, batch_size=50, remove_columns=['context', 'question'])
        test_dataset.save_to_disk(os.path.join(tokenized_dir, "test"))
        logger.info("Tokenized and saved test dataset")

    return train_dataset, val_dataset, test_dataset

# Load and assign tokenized datasets globally
processed_train_dataset, processed_val_dataset, processed_test_dataset = load_tokenized_datasets()

2025-07-24 14:56:52,002 - INFO - Loaded tokenized training dataset
2025-07-24 14:56:52,009 - INFO - Loaded tokenized validation dataset


Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

2025-07-24 14:56:53,156 - INFO - Loaded tokenized test dataset


In [14]:
# Log dataset sizes
logger.info(f"Train dataset size: {len(processed_train_dataset)}")
logger.info(f"Validation dataset size: {len(processed_val_dataset)}")
logger.info(f"Test dataset size: {len(processed_test_dataset)}")

2025-07-22 16:32:54,490 - INFO - Train dataset size: 80000
2025-07-22 16:32:54,490 - INFO - Validation dataset size: 10000
2025-07-22 16:32:54,490 - INFO - Test dataset size: 10000


In [15]:
# Clean datasets
for ds_name in ["processed_train_dataset", "processed_val_dataset", "processed_test_dataset"]:
    if ds_name in globals():
        ds = globals()[ds_name]
        cleaned = ds.remove_columns([c for c in ds.column_names if c not in ["input_ids", "attention_mask", "labels"]])
        globals()[ds_name] = cleaned


In [16]:
# Evaluation Metrics
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if isinstance(predictions, tuple):
        predictions = predictions[0]
    predictions = np.clip(predictions, 0, tokenizer.vocab_size - 1)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels[labels == -100] = tokenizer.pad_token_id
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    ref_tokens = [[nltk.word_tokenize(ref)] for ref in decoded_labels]
    pred_tokens = [nltk.word_tokenize(pred) for pred in decoded_preds]
    smoothie = SmoothingFunction().method4
    bleu1 = corpus_bleu(ref_tokens, pred_tokens, weights=(1, 0, 0, 0), smoothing_function=smoothie)
    bleu2 = corpus_bleu(ref_tokens, pred_tokens, weights=(0.5, 0.5, 0, 0), smoothing_function=smoothie)
    bleu3 = corpus_bleu(ref_tokens, pred_tokens, weights=(0.33, 0.33, 0.33, 0), smoothing_function=smoothie)
    bleu4 = corpus_bleu(ref_tokens, pred_tokens, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothie)
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    rouge_l = sum(
        scorer.score(ref, pred)['rougeL'].fmeasure
        for ref, pred in zip(decoded_labels, decoded_preds)
    ) / len(decoded_labels)
    meteor = sum(
        meteor_score([nltk.word_tokenize(ref)], nltk.word_tokenize(pred))
        for ref, pred in zip(decoded_labels, decoded_preds)
    ) / len(decoded_labels)
    try:
        P, R, F1 = score(decoded_preds, decoded_labels, lang="en", verbose=False)
        bertscore = F1.mean().item()
    except Exception:
        bertscore = 0.0
    return {
        "bleu-1": bleu1, "bleu-2": bleu2, "bleu-3": bleu3, "bleu-4": bleu4,
        "rouge-l": rouge_l, "meteor": meteor, "bertscore": bertscore
    }

In [18]:
# Custom Early Stopping Callback
class CustomEarlyStoppingCallback(EarlyStoppingCallback):
    def __init__(self, early_stopping_patience, min_delta=0.01):
        super().__init__(early_stopping_patience=early_stopping_patience)
        self.min_delta = min_delta
        self.best_metric = float('inf')
        self.early_stopping_patience_counter = 0

    def on_train_begin(self, args, state, control, **kwargs):
        if args.load_best_model_at_end:
            assert args.metric_for_best_model is not None, (
                "EarlyStoppingCallback requires metric_for_best_model to be defined when load_best_model_at_end=True"
            )
        assert args.eval_strategy != "no", (
            "EarlyStoppingCallback requires eval_strategy to be 'steps' or 'epoch'"
        )
        logger.info("Initialized CustomEarlyStoppingCallback")

    def on_evaluate(self, args, state, control, metrics, **kwargs):
        eval_loss = metrics.get('eval_loss', float('inf'))
        if self.best_metric == float('inf') or eval_loss < self.best_metric - self.min_delta:
            self.best_metric = eval_loss
            self.early_stopping_patience_counter = 0
        else:
            self.early_stopping_patience_counter += 1
        if self.early_stopping_patience_counter >= self.early_stopping_patience:
            logger.info(f"Early stopping triggered after {self.early_stopping_patience} evaluations with eval_loss={eval_loss}")
            control.should_training_stop = True




In [19]:
# Save trial results helper
def save_trial_results(study, output_dir):
    trials_data = []
    for trial in study.trials:
        trial_data = {
            'trial_number': trial.number,
            'eval_loss': trial.value if trial.value is not None else float('inf'),
            'state': str(trial.state),
            **trial.params
        }
        trials_data.append(trial_data)
    best_trial_data = {
        'trial_number': study.best_trial.number,
        'eval_loss': study.best_trial.value,
        'state': 'BEST',
        **study.best_params
    }
    trials_data.append(best_trial_data)
    output_path = os.path.join(output_dir, 'optuna_trials.csv')
    trials_df = pd.DataFrame(trials_data)
    mode = 'a' if os.path.exists(output_path) else 'w'
    trials_df.to_csv(output_path, index=False, mode=mode, header=not os.path.exists(output_path))
    logger.info("Saved trial results to optuna_trials.csv")

In [20]:
# Objective function for Optuna tuning
def objective(trial):
    torch.cuda.empty_cache()
    gc.collect()
    
    # Normalize and validate model path
    model_path = os.path.abspath(os.path.normpath(MODEL_PATH))
    if not os.path.exists(model_path):
        logger.error(f"Model path does not exist: {model_path}")
        raise FileNotFoundError(f"Model path does not exist: {model_path}")

    # Load model
    try:
        model = BartForConditionalGeneration.from_pretrained(
            model_path,
            local_files_only=True,
            use_safetensors=True,
            trust_remote_code=True
        )
    except Exception as e:
        logger.error(f"Failed to load model from {model_path}: {e}")
        raise

    model.to(device)
    model.gradient_checkpointing_enable()
    logger.info(f"Gradient checkpointing enabled: {model.is_gradient_checkpointing}")
    generation_config = model.generation_config
    generation_config.no_repeat_ngram_size = 3
    generation_config.min_length = 5
    generation_config.max_length = 64
    generation_config.num_beams = 3

    learning_rate = trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True)
    weight_decay = trial.suggest_float("weight_decay", 0.01, 0.1, log=True)
    warmup_steps = trial.suggest_int("warmup_steps", 100, 500, step=50)
    lr_scheduler_type = trial.suggest_categorical("lr_scheduler_type", ["linear", "cosine"])

    logger.info(f"Trial {trial.number} parameters: learning_rate={learning_rate}, "
                f"weight_decay={weight_decay}, warmup_steps={warmup_steps}, "
                f"lr_scheduler_type={lr_scheduler_type}")

    # Log VRAM usage before training
    if torch.cuda.is_available():
        vram_used = torch.cuda.memory_allocated(device) / 1024**3
        vram_total = torch.cuda.get_device_properties(device).total_memory / 1024**3
        logger.info(f"VRAM usage before training (trial {trial.number}): {vram_used:.2f}GB / {vram_total:.2f}GB")

    trial_output_dir = os.path.join(OUTPUT_DIR, f"trial_{trial.number}")
    os.makedirs(trial_output_dir, exist_ok=True)

    training_args = Seq2SeqTrainingArguments(
        output_dir=trial_output_dir,
        num_train_epochs=5,
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        gradient_accumulation_steps=16,
        dataloader_num_workers=0,
        dataloader_pin_memory=torch.cuda.is_available(),
        lr_scheduler_type=lr_scheduler_type,
        learning_rate=learning_rate,
        warmup_steps=warmup_steps,
        remove_unused_columns=False,
        report_to=[],
        eval_strategy="steps",
        eval_steps=200,
        save_strategy="no",
        weight_decay=weight_decay,
        fp16=torch.cuda.is_available(),
        logging_strategy="steps",
        logging_steps=50,
        predict_with_generate=True,
        generation_max_length=64,
        generation_num_beams=3,
        load_best_model_at_end=False,
        group_by_length=True,
        skip_memory_metrics=True,
        disable_tqdm=True,
    )

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=processed_train_dataset,
        eval_dataset=processed_val_dataset,
        compute_metrics=None,
        data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, padding=True),
        callbacks=[CustomEarlyStoppingCallback(early_stopping_patience=3)]
    )

    # Train the model with mixed precision
    logger.info(f"Starting new training for trial {trial.number}")
    try:
        with torch.cuda.amp.autocast():
            trainer.train()
    except Exception as e:
        logger.error(f"Training failed for trial {trial.number}: {e}")
        raise
    
    # Evaluate and report
    eval_results = trainer.evaluate()
    torch.cuda.empty_cache()
    gc.collect()
    
    # Log VRAM after training
    if torch.cuda.is_available():
        vram_used = torch.cuda.memory_allocated(device) / 1024**3
        logger.info(f"VRAM usage after training: {vram_used:.2f}GB / {vram_total:.2f}GB")
    
    for log in trainer.state.log_history:
        if 'eval_loss' in log:
            step = log.get("step", 0)
            trial.report(log['eval_loss'], step=step)
            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()
    
    return eval_results["eval_loss"]

In [21]:
# Run Optuna hyperparameter search with Bayesian optimization
study_name = "bart_question_generation"
storage_url = f"sqlite:///{os.path.join(OUTPUT_DIR, 'optuna_study.db')}"
db_path = os.path.join(OUTPUT_DIR, "optuna_study.db")
if os.path.exists(db_path):
    try:
        conn = sqlite3.connect(db_path)
        conn.close()
        os.remove(db_path)
        logger.info("Deleted existing Optuna database to start fresh.")
    except (PermissionError, sqlite3.OperationalError) as e:
        logger.warning(f"Could not delete existing database {db_path}: {e}. Reusing existing database.")

try:
    study = optuna.create_study(
        study_name=study_name,
        storage=storage_url,
        direction="minimize",
        sampler=TPESampler(seed=42),  # Bayesian optimization
        pruner=MedianPruner(n_warmup_steps=2),
        load_if_exists=True
    )
    study.optimize(objective, n_trials=15)
    save_trial_results(study, OUTPUT_DIR)
    best_params = study.best_params
    with open(os.path.join(OUTPUT_DIR, "best_params.json"), "w") as f:
        json.dump(best_params, f, indent=4)
    logger.info(f"Best hyperparameters: {best_params}")
    logger.info(f"Best objective value (eval_loss): {study.best_value}")

    # Clean up non-best trials
    best_trial_dir = os.path.join(OUTPUT_DIR, f"trial_{study.best_trial.number}")
    for trial_dir in os.listdir(OUTPUT_DIR):
        if trial_dir.startswith("trial_") and trial_dir != os.path.basename(best_trial_dir):
            try:
                shutil.rmtree(os.path.join(OUTPUT_DIR, trial_dir))
                logger.info(f"Deleted non-best trial directory: {trial_dir}")
            except Exception as e:
                logger.warning(f"Failed to delete trial directory {trial_dir}: {e}")
except Exception as e:
    logger.error(f"Optuna optimization or file saving failed: {e}")
    raise
finally:
    if 'storage' in locals():
        del storage
    logger.info("Closed Optuna storage connections.")

[I 2025-07-22 16:36:30,004] A new study created in RDB with name: bart_question_generation
2025-07-22 16:36:36,274 - INFO - Gradient checkpointing enabled: True
2025-07-22 16:36:36,304 - INFO - Trial 0 parameters: learning_rate=1.827226177606625e-05, weight_decay=0.08927180304353628, warmup_steps=400, lr_scheduler_type=linear
2025-07-22 16:36:36,306 - INFO - VRAM usage before training (trial 0): 1.51GB / 7.96GB
2025-07-22 16:36:36,431 - INFO - Starting new training for trial 0
C:\Users\User\AppData\Local\Temp\ipykernel_3680\2499197808.py:93: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
2025-07-22 16:36:46,750 - INFO - Initialized CustomEarlyStoppingCallback


{'loss': 2.7963, 'grad_norm': 8.041173934936523, 'learning_rate': 2.0556294498074535e-06, 'epoch': 0.02}
{'loss': 2.6685, 'grad_norm': 6.349306106567383, 'learning_rate': 4.3396621718157345e-06, 'epoch': 0.04}
{'loss': 2.5531, 'grad_norm': 5.658001899719238, 'learning_rate': 6.623694893824016e-06, 'epoch': 0.06}
{'loss': 2.5554, 'grad_norm': 5.723142623901367, 'learning_rate': 8.862046961392131e-06, 'epoch': 0.08}
{'eval_loss': 2.371518850326538, 'eval_runtime': 109.2292, 'eval_samples_per_second': 91.551, 'eval_steps_per_second': 45.775, 'epoch': 0.08}
{'loss': 2.5198, 'grad_norm': 10.649038314819336, 'learning_rate': 1.1146079683400413e-05, 'epoch': 0.1}
{'loss': 2.46, 'grad_norm': 9.487222671508789, 'learning_rate': 1.3430112405408695e-05, 'epoch': 0.12}
{'loss': 2.4502, 'grad_norm': 11.642400741577148, 'learning_rate': 1.5714145127416976e-05, 'epoch': 0.14}
{'loss': 2.4304, 'grad_norm': 6.292455196380615, 'learning_rate': 1.7998177849425257e-05, 'epoch': 0.16}
{'eval_loss': 2.28006

2025-07-22 18:54:21,414 - INFO - Early stopping triggered after 3 evaluations with eval_loss=2.091510534286499


{'eval_loss': 2.091510534286499, 'eval_runtime': 109.8784, 'eval_samples_per_second': 91.01, 'eval_steps_per_second': 45.505, 'epoch': 1.3599999999999999}
{'train_runtime': 8254.6836, 'train_samples_per_second': 48.457, 'train_steps_per_second': 1.514, 'train_loss': 2.2362316086713006, 'epoch': 1.3599999999999999}


2025-07-22 18:56:35,343 - INFO - Early stopping triggered after 3 evaluations with eval_loss=2.091402530670166


...

2025-07-24 03:12:12,208 - INFO - Early stopping triggered after 3 evaluations with eval_loss=2.100008249282837


{'eval_loss': 2.100008249282837, 'eval_runtime': 118.5215, 'eval_samples_per_second': 84.373, 'eval_steps_per_second': 42.186, 'epoch': 1.12}
{'train_runtime': 7088.6564, 'train_samples_per_second': 56.428, 'train_steps_per_second': 1.763, 'train_loss': 2.2784713472638813, 'epoch': 1.12}


2025-07-24 03:15:31,921 - INFO - Early stopping triggered after 3 evaluations with eval_loss=2.101543426513672


{'eval_loss': 2.101543426513672, 'eval_runtime': 198.3846, 'eval_samples_per_second': 50.407, 'eval_steps_per_second': 25.204, 'epoch': 1.12}


2025-07-24 03:15:32,287 - INFO - VRAM usage after training: 4.43GB / 7.96GB
[I 2025-07-24 03:15:32,311] Trial 14 pruned. 
2025-07-24 03:15:32,345 - INFO - Saved trial results to optuna_trials.csv
2025-07-24 03:15:32,350 - INFO - Best hyperparameters: {'learning_rate': 1.2853916978930139e-05, 'weight_decay': 0.011430983876313222, 'warmup_steps': 450, 'lr_scheduler_type': 'cosine'}
2025-07-24 03:15:32,355 - INFO - Best objective value (eval_loss): 2.0840227603912354
2025-07-24 03:15:32,360 - INFO - Deleted non-best trial directory: trial_0
2025-07-24 03:15:32,360 - INFO - Deleted non-best trial directory: trial_10
2025-07-24 03:15:32,361 - INFO - Deleted non-best trial directory: trial_11
2025-07-24 03:15:32,362 - INFO - Deleted non-best trial directory: trial_12
2025-07-24 03:15:32,363 - INFO - Deleted non-best trial directory: trial_13
2025-07-24 03:15:32,363 - INFO - Deleted non-best trial directory: trial_14
2025-07-24 03:15:32,364 - INFO - Deleted non-best trial directory: trial_2
2

In [23]:
# Train final model with best hyperparameters
logger.info("Training final model with best hyperparameters")
best_params = study.best_params
model = BartForConditionalGeneration.from_pretrained(MODEL_PATH, trust_remote_code=True, use_safetensors=True)
model.to(device)
model.gradient_checkpointing_enable()
model.generation_config.no_repeat_ngram_size = 3
model.generation_config.min_length = 5

# Log VRAM usage before final training
if torch.cuda.is_available():
    vram_used = torch.cuda.memory_allocated(device) / 1024**3
    vram_total = torch.cuda.get_device_properties(device).total_memory / 1024**3
    logger.info(f"VRAM usage before final training: {vram_used:.2f}GB / {vram_total:.2f}GB")

# Define training arguments with best parameters
training_args = Seq2SeqTrainingArguments(
    output_dir=os.path.join(OUTPUT_DIR, "final_model"),
    num_train_epochs=5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=16,
    dataloader_num_workers=0,
    dataloader_pin_memory=torch.cuda.is_available(),
    lr_scheduler_type=best_params["lr_scheduler_type"],
    learning_rate=best_params["learning_rate"],
    warmup_steps=best_params["warmup_steps"],
    weight_decay=best_params["weight_decay"],
    fp16=torch.cuda.is_available(),
    logging_strategy="steps",
    logging_steps=50,
    eval_strategy="epoch",  # Changed from evaluation_strategy
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    predict_with_generate=True,
    generation_max_length=64,
    generation_num_beams=3,
    group_by_length=True,
    skip_memory_metrics=True,
    disable_tqdm=True,
)

# Initialize data collator
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, padding=True)

# Initialize trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=processed_train_dataset,
    eval_dataset=processed_val_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    callbacks=[CustomEarlyStoppingCallback(early_stopping_patience=3)]
)

# Train the model
try:
    logger.info("Starting final training")
    with torch.amp.autocast('cuda'):  # Updated to use correct autocast API
        trainer.train()
    logger.info("Final training completed successfully")
except Exception as e:
    logger.error(f"Final training failed: {e}")
    raise


# Log VRAM usage after training
if torch.cuda.is_available():
    vram_used = torch.cuda.memory_allocated(device) / 1024**3
    logger.info(f"VRAM usage after final training: {vram_used:.2f}GB / {vram_total:.2f}GB")

# Clean up memory
torch.cuda.empty_cache()
gc.collect()


2025-07-24 03:18:04,084 - INFO - Training final model with best hyperparameters
2025-07-24 03:18:05,551 - INFO - VRAM usage before final training: 5.95GB / 7.96GB
2025-07-24 03:18:05,734 - INFO - Starting final training
2025-07-24 03:18:16,375 - INFO - Initialized CustomEarlyStoppingCallback


{'loss': 2.8115, 'grad_norm': 7.2612762451171875, 'learning_rate': 1.285391697893014e-06, 'epoch': 0.02}
{'loss': 2.7058, 'grad_norm': 6.481954574584961, 'learning_rate': 2.713604695551918e-06, 'epoch': 0.04}
{'loss': 2.5877, 'grad_norm': 5.4879150390625, 'learning_rate': 4.141817693210823e-06, 'epoch': 0.06}
{'loss': 2.5881, 'grad_norm': 6.2297821044921875, 'learning_rate': 5.541466430916549e-06, 'epoch': 0.08}
{'loss': 2.5418, 'grad_norm': 8.759868621826172, 'learning_rate': 6.969679428575454e-06, 'epoch': 0.1}
{'loss': 2.4848, 'grad_norm': 9.387632369995117, 'learning_rate': 8.397892426234358e-06, 'epoch': 0.12}
{'loss': 2.4736, 'grad_norm': 13.297496795654297, 'learning_rate': 9.826105423893262e-06, 'epoch': 0.14}
{'loss': 2.4506, 'grad_norm': 5.881244659423828, 'learning_rate': 1.1254318421552166e-05, 'epoch': 0.16}
{'loss': 2.4455, 'grad_norm': 6.311375141143799, 'learning_rate': 1.268253141921107e-05, 'epoch': 0.18}
{'loss': 2.4191, 'grad_norm': 12.817657470703125, 'learning_rat

2025-07-24 05:12:17,566 - INFO - Using default tokenizer.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'eval_loss': 2.1029350757598877, 'eval_bleu-1': 0.2700138367266258, 'eval_bleu-2': 0.148682834700573, 'eval_bleu-3': 0.10192163977284457, 'eval_bleu-4': 0.07242951093638948, 'eval_rouge-l': 0.2205406619320127, 'eval_meteor': 0.23111950149719418, 'eval_bertscore': 0.8767882585525513, 'eval_runtime': 2078.7477, 'eval_samples_per_second': 4.811, 'eval_steps_per_second': 2.405, 'epoch': 1.0}
{'loss': 2.1202, 'grad_norm': 5.572562217712402, 'learning_rate': 1.1920584233401736e-05, 'epoch': 1.02}
{'loss': 2.1231, 'grad_norm': 8.109247207641602, 'learning_rate': 1.187663767737425e-05, 'epoch': 1.04}
{'loss': 2.0864, 'grad_norm': 5.420586585998535, 'learning_rate': 1.1831765079852086e-05, 'epoch': 1.06}
{'loss': 2.1186, 'grad_norm': 7.683208465576172, 'learning_rate': 1.1785974065850561e-05, 'epoch': 1.08}
{'loss': 2.1085, 'grad_norm': 9.568700790405273, 'learning_rate': 1.173927241644771e-05, 'epoch': 1.1}
{'loss': 2.1213, 'grad_norm': 4.8004841804504395, 'learning_rate': 1.1691668067462093e

2025-07-24 07:10:10,170 - INFO - Using default tokenizer.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

2025-07-24 12:55:46,301 - INFO - Using default tokenizer.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'eval_loss': 2.0252857208251953, 'eval_bleu-1': 0.27332187196680857, 'eval_bleu-2': 0.15301182716600392, 'eval_bleu-3': 0.10548378318198912, 'eval_bleu-4': 0.07516052867158196, 'eval_rouge-l': 0.23357904550707687, 'eval_meteor': 0.2461827601471226, 'eval_bertscore': 0.8796655535697937, 'eval_runtime': 2139.2749, 'eval_samples_per_second': 4.674, 'eval_steps_per_second': 2.337, 'epoch': 5.0}


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].
2025-07-24 12:56:23,212 - INFO - Final training completed successfully
2025-07-24 12:56:23,222 - INFO - VRAM usage after final training: 4.43GB / 7.96GB


{'train_runtime': 34686.8464, 'train_samples_per_second': 11.532, 'train_steps_per_second': 0.36, 'train_loss': 2.013610836791992, 'epoch': 5.0}


1615

In [46]:
# Save final model and tokenizer
reload_model_dir = os.path.join(final_model_dir, "reload_model")
os.makedirs(reload_model_dir, exist_ok=True)
try:
    model.save_pretrained(reload_model_dir, safe_serialization=True)
    tokenizer.save_pretrained(reload_model_dir)
    necessary_files = [
        "model.safetensors",
        "config.json",
        "tokenizer_config.json",
        "vocab.json",
        "merges.txt",
        "special_tokens_map.json"
    ]
    for file in os.listdir(reload_model_dir):
        if file not in necessary_files:
            os.remove(os.path.join(reload_model_dir, file))
    logger.info(f"Saved model and tokenizer to {reload_model_dir}")
except Exception as e:
    logger.error(f"Failed to save model/tokenizer to {reload_model_dir}: {e}")
    raise

2025-07-24 16:09:04,593 - INFO - Saved model and tokenizer to D:\A_CSE499\outputLarge_B_phase2\final_model\reload_model


In [49]:
import os
import time
from tqdm import tqdm
import pandas as pd
import numpy as np
import torch

# Set output directory
OUTPUT_DIR = r"D:\A_CSE499\outputLarge_B_phase2"  # Use raw string to handle backslashes

# Evaluate on test set and save results
model.eval()
torch.cuda.empty_cache()

logger.info("Starting prediction on test set...")
start = time.time()
test_results = trainer.predict(processed_test_dataset)
end = time.time()
logger.info(f"Prediction completed in {(end - start)/60:.2f} minutes.")

# Ensure output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)  # Create directory if it doesn't exist

# Save test metrics
test_metrics = test_results.metrics
test_results_df = pd.DataFrame([test_metrics])
test_results_path = os.path.join(OUTPUT_DIR, "test_results.csv")
try:
    test_results_df.to_csv(test_results_path, index=False)
    logger.info(f"Saved test results to {test_results_path}")
except Exception as e:
    logger.error(f"Failed to save test results: {e}")
    raise

# Handle predictions
test_preds = test_results.predictions[0] if isinstance(test_results.predictions, tuple) else test_results.predictions
test_preds = np.clip(test_preds, 0, tokenizer.vocab_size - 1)

# Decode with progress bars
logger.info("Decoding predictions...")
decoded_preds = tokenizer.batch_decode(
    list(tqdm(test_preds, desc="Decoding Predictions")), skip_special_tokens=True
)

label_ids = np.clip(test_results.label_ids, 0, tokenizer.vocab_size - 1)
logger.info("Decoding references...")
decoded_refs = tokenizer.batch_decode(
    list(tqdm(label_ids, desc="Decoding References")), skip_special_tokens=True
)

# Reconstruct context from input_ids
logger.info("Decoding original context inputs...")
decoded_contexts = [
    tokenizer.decode(sample["input_ids"], skip_special_tokens=True)
    for sample in tqdm(processed_test_dataset, desc="Decoding Contexts")
]

# Save predictions and references
pred_ref_df = pd.DataFrame({
    "context": decoded_contexts,
    "predicted_question": decoded_preds,
    "reference_question": decoded_refs
})
pred_ref_path = os.path.join(OUTPUT_DIR, "test_predictions.csv")
try:
    pred_ref_df.to_csv(pred_ref_path, index=False)
    logger.info(f"Saved test predictions to {pred_ref_path}")
except Exception as e:
    logger.error(f"Failed to save test predictions: {e}")
    raise

2025-07-24 19:36:56,669 - INFO - Starting prediction on test set...
2025-07-24 20:19:05,396 - INFO - Using default tokenizer.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
2025-07-24 20:20:19,691 - INFO - Prediction completed in 43.38 minutes.
2025-07-24 20:20:19,696 - INFO - Saved test results to D:\A_CSE499\outputLarge_B_phase2\test_results.csv
2025-07-24 20:20:19,696 - INFO - Decoding predictions...
Decoding Predictions: 100%|██████████| 10000/10000 [00:00<00:00, 999643.45it/s]
2025-07-24 20:20:19,865 - INFO - Decoding references...
Decoding References: 100%|██████████| 10000/10000 [00:00<00:00, 1466796.29it/s]
2025-07-24 20:20:20,032 - INFO - Decoding original context inputs...
Decoding Contexts: 100%|██████████| 10000/10000 [00:01<00:00, 5434.00it/s]
2025

In [51]:
try:
    num_examples = min(5, len(decoded_preds), len(decoded_refs), len(decoded_contexts))
    logger.info("\nExample Predictions and References:")
    for i in range(num_examples):
        logger.info(f"\nExample {i+1}:")
        logger.info(f"Context: {decoded_contexts[i]}")
        logger.info(f"Prediction: {decoded_preds[i]}")
        logger.info(f"Reference: {decoded_refs[i]}")
except Exception as e:
    logger.error(f"Failed to print example predictions: {e}")
    raise

2025-07-24 20:33:57,316 - INFO - 
Example Predictions and References:
2025-07-24 20:33:57,316 - INFO - 
Example 1:
2025-07-24 20:33:57,317 - INFO - Context: we used it on our ford f250, it runs off the truck battery — lifts 1000lbs easily probably more, works great, goes flat with ground and brings whatever level w truck bed.
2025-07-24 20:33:57,318 - INFO - Prediction: What may happen after seeing JM?
2025-07-24 20:33:57,318 - INFO - Reference: How would you feel if you two decided to be more than friends?
2025-07-24 20:33:57,319 - INFO - 
Example 2:
2025-07-24 20:33:57,319 - INFO - Context: Michelle Trachtenberg: Michelle Christine Trachtenberg (born October 11, 1985) is an American actress. She is known for portraying Nona F. Mecklenberg in"The Adventures of Pete & Pete"(1994 — 96), Dawn Summers in"Buffy the Vampire Slayer"(2000 —
2025-07-24 20:33:57,320 - INFO - Prediction: Are Erin Wiedner and Jessie Bear both directors?
2025-07-24 20:33:57,320 - INFO - Reference: Which director i